# SHEEP: <br> a Homomorphic Encryption Evaluation Platform

Nick Barlow<br>
Adria Gascon<br>
Tomas Lazauskas<br>
Oliver Strickson<br>

<img src="static/turing_logo.webp" alt="Turing Logo" width="15%" align="left" />

<br clear="all" />

**7th WAHC, November 11, 2019, Hilton Metropole, London, UK**

Acknowledgements: Matt Kusner, Amartya Sanyal, Varun Kanade, Mariana Raykova, Shai Halevi, Kim Laine

## Who we are

<div>
<img src="static/adria.jpg" alt="Adria" width="15%" style="float:left;padding:5px"/>
<br clear="all" />
Adria Gascon, (former) Research Fellow, The Alan Turing Institute (now Google)
</div>

<div>
<img src="static/nick.jpg" alt="Nick" width="15%" style="float:left;padding:5px"/>
<img src="static/tomas.jpg" alt="Tomas" width="15%" style="float:left;padding:5px"/>
<img src="static/oliver.jpg" alt="Oliver" width="15%" style="float:left;padding:5px"/>
<br clear="all" />
Nick, Tomas, Oliver: Research Engineering, The Alan Turing Institute
</div>


## Research Engineering at the Alan Turing Institute

 - The Alan Turing Institute is the UK's national centre for data science and AI
 - Research Engineering is a team of research **software engineers** and **data scientists**
  - Goal is to help connect cutting-edge research to real-world problems and applications

## SHEEP

SHEEP is a Homomorphic Encryption Evaluation Platform

- General aim of Homomorphic encryption: Protect data while **computing on it**

<img src='static/he_eqn.png' alt='Homomorphic encryption' />

## SHEEP

- Is HE practical?
- Many different libraries and versions of these
- Rapidly changing field
- Need for **practical** way to **compare libraries** for a **given application**

## Inside SHEEP

<img src='static/SHEEP_diagram.png' alt="SHEEP diagram" width="50%"/>

## SHEEP features
 - Easily run performance benchmarks 
   - high level language
 - Compare libraries
   - easy to write/maintain wrapper
   - exposes parameters offered by the libraries
 - Reporting
   - runtime, ciphertext size, parallelization

## The SHEEP Circuit Description Language

To compute `total = (a + b) + c`

```
INPUTS a b c
OUTPUTS total
a b ADD d
d c ADD total
```

## The SHEEP language and supported libraries

<img src="static/sheep_contexts.gif" alt="contexts" />


## A quick tour of SHEEP



First some quick setup - set an environment variable so we can find our pre-made circuit files, then import the `sheep_client` module which contains methods to interact with the SHEEP API:

In [ ]:
import os
import uuid
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)
# import the sheep_client module
from pysheep import sheep_client

Tell the SHEEP server that we want to start a new job:         

In [ ]:
sheep_client.new_job()

List the available contexts:

In [ ]:
sheep_client.get_available_contexts()

Lets do a test with LibPaillier:

In [ ]:
sheep_client.set_context("LP")

List the available input types:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("LP")
sheep_client.get_available_input_types()

Let's use uint16_t:

In [ ]:
sheep_client.set_input_type("uint16_t")

Now let's give it a circuit.  There are plenty of simple circuits in the ```benchmark_inputs/``` directory.  We need to provide the full path to the circuit though so we'll use the SHEEP_HOME we defined at the start.

In [ ]:
r=sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))
print(sheep_client.get_circuit()['content']['circuit'])

We can see the names of the "inputs" that this circuit expects:

In [ ]:
sheep_client.get_inputs()

Now we can set values for those inputs.  The circuit we've chosen just adds the two inputs together, so lets do 55+44 (and hope we get the answer 99 at the end!)  Note that we have to write the inputs as lists, as potentially we can do more than one calculation at the same time (known as SIMD)).

In [ ]:
sheep_client.set_inputs({"input_0":[55],"input_1":[44]})

We have set the context, the input_type, the circuit_file and the input values - we should be ready to go!  Let's print out the configuration, and then check that we are fully configured.

In [ ]:
sheep_client.get_config()

OK, let's execute the test!

In [ ]:
sheep_client.run_job()

The SHEEP server just used our selected "context" (i.e. HE library) to encrypt our inputs, evaluate our circuit, and decrypt the outputs.  It also then evaluated the same circuit "in the clear" (i.e. with no encryption) so that we can check we got the right answer.   The result, including the output values, the processing times, and the clear-text check, are available as a json object.

In [ ]:
r=sheep_client.get_results()
from pprint import pprint
pprint(r)

So we can see that 
 * we got the answer "99" (which is 55+44, the sum of our two inputs) 
 * all the output values we got (just one in this case) matched the result from the "cleartext" context.
 * from the timings (which are in microseconds), the encryption and decryption were fast, but evaluating the circuit took some time.

Lets do another circuit - this time with a COMPARE gate which returns 1 if the first input is greater than the second input, and zero otherwise:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("TFHE")
sheep_client.set_input_type("int8_t")
sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-COMPARE-1.sheep"))
sheep_client.set_inputs({"input_0": [11], "input_1": [10]})
sheep_client.run_job()
sheep_client.get_results()

## Using "slots"

Many HE algorithms can operate on vectors of inputs simultaneously (SIMD operations).  This is supported in SHEEP as follows:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int8_t")
sheep_client.get_nslots()

We can now introduce a new operation, ROTATE to shift the elements of the slots around:

In [ ]:
sheep_client.set_circuit_text("INPUTS in \n CONST_INPUTS shift \n OUTPUTS out \n in shift ROTATE out\n")
sheep_client.set_inputs({"in": [1,2,3,4]})
sheep_client.set_const_inputs({"shift": 1})
sheep_client.run_job()
sheep_client.get_results()

We only got four slots with that set of parameters in HElib, but we can try a different set:

In [ ]:
sheep_client.set_parameters({"BaseParamSet": 2})
sheep_client.get_nslots()

For those libraries that do not support SIMD operations, we still allow slots (with "nslots" as a configurable parameter), but the calculation is of course slower (by a factor nslots).

In [ ]:
sheep_client.set_context("TFHE")
sheep_client.get_nslots()

In [ ]:
sheep_client.set_parameters({"NumSlots": 100})
sheep_client.get_nslots()

SEAL_BFV does support SIMD, and the default parameters give us a large number of slots:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("SEAL_BFV")
sheep_client.set_input_type("int8_t")
sheep_client.get_nslots()

If we don't need all those slots. If we specify fewer, the server will fill the rest up with a repeating pattern of the specified inputs, so that ROTATE operations will still be correct.


Now lets try a simple "MULTIPLY" across five slots.

In [ ]:
sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-MULTIPLY-1.sheep"))
sheep_client.set_inputs({"input_0":[1,2,3,4,5],"input_1":[2,4,6,8,10]})
sheep_client.run_job()
sheep_client.get_results()

## Low-level benchmarks

The first set of benchmarks to be run corresponds to circuits composed of only one type of gate, with various "depths" (i.e. numbers of nested gates).

For example, circuit-ADD-3.sheep contains three nested "ADD" gates: output=((a+b)+c)+d.



We perform several "scans" of parameter space. For each scan, some functions in benchmark_utils will be run to configure and run the HE libraries, and then we visualize the results.

We will show a shortened version of the first of these scans here as an example:

In [ ]:
import re
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from pysheep.low_level_benchmarks import params_for_level, run_circuit

### Scan 1
Set parameters for the "leveled" libraries (HElib_Fp, SEAL_BFV and PALISADE) to be able to support multiplicative depth of 10, then compare the timings for different depths of ADD and MULTIPLY.

In [ ]:
CIRCUIT_FILE_DIR = os.path.join(os.environ["SHEEP_HOME"],"benchmark_inputs","low_level","circuits")
## give a unique scan_id so that we can retrieve the results later on
scan_id = str(uuid.uuid4())
## look at bool inputs
input_type = "bool"
## start looping over gates, depths, and contexts
for gate in ["ADD","MULTIPLY"]:
    for context in ["HElib_Fp","SEAL_BFV","PALISADE"]:
        params = params_for_level(context,10)
        for d in range(1,6):
            circuit_file = CIRCUIT_FILE_DIR+"/circuit-"+gate+"-"+str(d)+".sheep"
            print("Doing benchmark for %s %s %s %i" % (context,gate,input_type,d))
            uploaded_OK = run_circuit(circuit_file, input_type, context, params,"serial",scan_id,timeout=60)
            if not uploaded_OK:
                print("Problem running test or uploading result")
if uploaded_OK:
    print("Finished OK")

### Tabulating and plotting the results

We use matplotlib and pandas to plot the results, and store the results in an sqlite file:

In [ ]:
from pysheep.database import BenchmarkMeasurement, session
from pysheep.benchmark_utils import levels_for_params
from pysheep.low_level_benchmarks import get_gate_name_from_circuit_name


In [ ]:
def get_levels_for_row(benchmark_measurement):
    ## do the reverse lookup of params_for_level
    context = benchmark_measurement.context
    param_dict = {}
    for param in benchmark_measurement.parameters:
        param_dict[param.param_name] = param.param_value
    return levels_for_params(benchmark_measurement.context,param_dict)

def get_depth_for_row(benchmark_measurement):
    ## parse the circuit filename and see how many nested gates there were 
    return re.search("-([\d]+).sheep",benchmark_measurement.circuit_name).groups()[0]


def get_execution_time_for_row(benchmark_measurement):
    for timing in benchmark_measurement.timings:
        if timing.timing_name == "evaluation":
            return timing.timing_value

def fill_row(benchmark_measurement, columns):
    columns["context"].append(benchmark_measurement.context)
    columns["gate"].append(get_gate_name_from_circuit_name(benchmark_measurement.circuit_name))
    columns["input_bitwidth"].append(benchmark_measurement.input_bitwidth)
    columns["input_signed"].append(benchmark_measurement.input_signed)
    if "level" in columns.keys():
        columns["level"].append(get_levels_for_row(benchmark_measurement))
    if "depth" in columns.keys():
        columns["depth"].append(get_depth_for_row(benchmark_measurement))
    columns["execution_time"].append(get_execution_time_for_row(benchmark_measurement))
    return columns

In [ ]:
rows_scan1 = session.query(BenchmarkMeasurement).filter_by(scan_id=scan_id).all()
dict_scan1 = {"context":[],"gate":[],"input_bitwidth":[],"input_signed":[],"depth":[], "execution_time":[]}
for row in rows_scan1:
    dict_scan1 = fill_row(row,dict_scan1)
df_scan1 = pd.DataFrame(dict_scan1)
df_scan1.head()

Now lets write a function to plot execution time vs depth for each gate type:

In [ ]:
def execution_time_vs_x_cat(input_dataframe, gate_name, bitwidth, is_signed, libs_list, x_var="context",category_var=None):
    ## plot execution time for different contexts vs category var (e.g. "depth", "levels")
    rows = input_dataframe[(input_dataframe.input_bitwidth==bitwidth) & 
                           (input_dataframe.gate==gate_name) & 
                           input_dataframe.context.isin(libs_list) & 
                           (input_dataframe.input_signed==is_signed)
                          ]
 #   return rows
    if category_var:
        return rows.drop_duplicates([x_var, category_var]).pivot(
            index=x_var, columns=category_var, values='execution_time')
    else:
        return rows.drop_duplicates([x_var]).pivot(
            index=x_var, values='execution_time')

In [ ]:
all_ops = set(df_scan1.gate)
bitwidth = 1
is_signed = False
for i, op in enumerate(all_ops):
    libs = ["HElib_Fp", "SEAL_BFV","PALISADE"]
    plot_scan1 = execution_time_vs_x_cat(df_scan1, op, bitwidth, is_signed, libs,"context","depth")
    axis = plt.subplot((len(all_ops) + 2 // 2) // 2, 2, i + 1)
#     axis.set_yscale("log")
    plot_scan1.plot(ax=axis, title=op + ' with bitwidth = ' + str(bitwidth), kind='bar', figsize=(15, 10), legend=False if i!=0 else True)
plt.tight_layout()
plt.show()

## Mid-level benchmark - Private Information Retrieval

One application of HE is to retrieve a data point from a database held elsewhere, without the database holder knowing which point is being requested. If Alice wants to query Bob's database, she simply encrypts an array full of zeros, except for a "1" in the position of the desired data point, and sends this ciphertext to Bob. He then performs homomorphic multiplication and addition, and sends the ciphertext back to Alice, who decrypts it to retrieve the data point she requested.

To demonstrate this on a very small scale, lets look at a "database" containing only two values: 123 and 456:

In [ ]:
from pysheep.mid_level_benchmarks import generate_pir_circuit, \
    generate_variance_circuit, generate_bitonic_sort_circuit, generate_gaussian_inputs
from pysheep import benchmark_utils
from pysheep import common_utils

sheep_client.new_job() 
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int16_t")

Now let's generate the circuit file for this simple PIR case with two values in the "database":

In [ ]:
circuit_file=generate_pir_circuit(2,[2])
sheep_client.set_circuit(circuit_file)
sheep_client.get_inputs()

"d_a_b_c" are the "database" values, and "s_x_y" are the "selector" values. We can set d_0_0_0 and d_0_1_0 to "123" and "456" respectively. Then, to select d_0_0_0 (which will hopefully be the value "123") from the database, we should set s_0_0 to 1 and s_0_1 to zero.

In [ ]:
sheep_client.set_inputs({"d_0_0_0": [123], "d_0_1_0": [456], "s_0_0": [1], "s_0_1": [0]})
sheep_client.run_job()

In [ ]:
sheep_client.get_results()

So we successfully got the output value "123", i.e. the first entry in the "database".

## Mid-level benchmark - Matrix vector multiplication

Lets do the following calculation:
\begin{equation*}
\begin{vmatrix}
1, 2 , 3 , 4\\
5 , 6 , 7 , 8\\
9, 10, 11, 12 \\
13, 14 , 15, 16
\end{vmatrix}
\begin{vmatrix}
1 \\
2 \\
3 \\
4
\end{vmatrix}
\end{equation*}

which should give us the answer {30, 70, 110, 150}

### a) The straightforward (but non-optimal) way

Essentially the each element of the output vector will be the dot product of the corresponding row of the matrix with the vector.  We can therefore do this in the same way as was demonstrated in the vector_dot_product notebook, i.e. component-wise multiplication followed by a sequence of rotations and additions.

The circuit will look like:

In [ ]:
circuit = """
INPUTS mrow_0 mrow_1 mrow_2 mrow_3 vec mask
CONST_INPUTS rotate_by_minus1 rotate_by_plus1 rotate_by_plus2 rotate_by_plus3
OUTPUTS output_vec 
# dot product of the first row with the vector
mrow_0 vec MULTIPLY prod_00
prod_00 rotate_by_minus1 ROTATE prod_01
prod_00 prod_01 ADD sum_00
prod_01 rotate_by_minus1 ROTATE prod_02
sum_00 prod_02 ADD sum_01
prod_02 rotate_by_minus1 ROTATE prod_03
sum_01 prod_03 ADD sum_0
# dot product of the second row with the vector
mrow_1 vec MULTIPLY prod_10
prod_10 rotate_by_minus1 ROTATE prod_11
prod_10 prod_11 ADD sum_10
prod_11 rotate_by_minus1 ROTATE prod_12
sum_10 prod_12 ADD sum_11
prod_12 rotate_by_minus1 ROTATE prod_13
sum_11 prod_13 ADD sum_1
# dot product of the third row with the vector
mrow_2 vec MULTIPLY prod_20
prod_20 rotate_by_minus1 ROTATE prod_21
prod_20 prod_21 ADD sum_20
prod_21 rotate_by_minus1 ROTATE prod_22
sum_20 prod_22 ADD sum_21
prod_22 rotate_by_minus1 ROTATE prod_23
sum_21 prod_23 ADD sum_2
# dot product of the third row with the vector
mrow_3 vec MULTIPLY prod_30
prod_30 rotate_by_minus1 ROTATE prod_31
prod_30 prod_31 ADD sum_30
prod_31 rotate_by_minus1 ROTATE prod_32
sum_30 prod_32 ADD sum_31
prod_32 rotate_by_minus1 ROTATE prod_33
sum_31 prod_33 ADD sum_3
# now we have four vectors, sum_0, sum_1, sum_2 and sum_3, where the first element is 
# the dot product of that row.  We need to isolate just this element, using mask, which is [1,0,0,0]
sum_0 mask MULTIPLY msum_0
sum_1 mask MULTIPLY msum_10
msum_10 rotate_by_plus1 ROTATE msum_1
sum_2 mask MULTIPLY msum_20
msum_20 rotate_by_plus2 ROTATE msum_2
sum_3 mask MULTIPLY msum_30
msum_30 rotate_by_plus3 ROTATE msum_3
# now we should have four vectors with one non-zero element each in the right place - need to sum them
msum_0 msum_1 ADD out_01
out_01 msum_2 ADD out_02
out_02 msum_3 ADD output_vec
"""

In [ ]:
sheep_client.new_job()
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int16_t")
sheep_client.set_parameters({"Levels":30})
sheep_client.set_circuit_text(circuit)
sheep_client.get_inputs()

In [ ]:
sheep_client.get_const_inputs()

In [ ]:
sheep_client.set_inputs({"mrow_0": [1,2,3,4], "mrow_1": [5,6,7,8], "mrow_2": [9,10,11,12], "mrow_3": [13,14,15,16],"vec": [1,2,3,4], "mask": [1,0,0,0]})
sheep_client.set_const_inputs({"rotate_by_minus1": -1, "rotate_by_plus1": 1, "rotate_by_plus2": 2, "rotate_by_plus3":3})
sheep_client.run_job()
                              

In [ ]:
sheep_client.get_results()

In [ ]:
benchmark_utils.upload_results("matrix_vector_mult_4x4x4")

Now lets do the same test with SEAL_BFV:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("SEAL_BFV")
sheep_client.set_input_type("int16_t")
sheep_client.set_circuit_text(circuit)
sheep_client.set_inputs({"mrow_0": [1,2,3,4], "mrow_1": [5,6,7,8], "mrow_2": [9,10,11,12], "mrow_3": [13,14,15,16],"vec": [1,2,3,4], "mask": [1,0,0,0]})
sheep_client.set_const_inputs({"rotate_by_minus1": -1, "rotate_by_plus1": 1, "rotate_by_plus2": 2, "rotate_by_plus3":3})
sheep_client.run_job()
benchmark_utils.upload_results("matrix_vector_mult_4x4x4")

In [ ]:
rows = session.query(BenchmarkMeasurement).filter_by(circuit_name="matrix_vector_mult_4x4x4").all()
df_dict = {"Context": [], "Gate": [], "Time": []}
for row in rows:
    timingdict = benchmark_utils.timing_per_gate_type(row.timings, circuit)
    for k,v in timingdict.items():
        df_dict["Context"].append(row.context)
        df_dict["Gate"].append(k)
        df_dict["Time"].append(v)
df = pd.DataFrame(df_dict)
df

In [ ]:
pivot_df = df.pivot(index='Context', columns='Gate', values='Time')
pivot_df.plot(kind='bar',stacked=True)

### b) A better way -  doing the same calculation with fewer ROTATEs.

The paper describing the [GAZELLE](https://eprint.iacr.org/2018/073.pdf) framework includes a clever method for speeding up matrix-vector multiplication with SIMD operations.
To minimize the number of intra-slot operations, we can express the matrix inputs as diagonal strips of the matrix.

In this case, the circuit will look like:

In [ ]:
new_circuit = """
INPUTS mstrip_0 mstrip_1 mstrip_2 mstrip_3 vec 
CONST_INPUTS rotate_minus1
OUTPUTS output_vec 
mstrip_0 vec MULTIPLY prod_0
vec rotate_minus1 ROTATE vec_r1
mstrip_1 vec_r1 MULTIPLY prod_1
vec_r1 rotate_minus1 ROTATE vec_r2
mstrip_2 vec_r2 MULTIPLY prod_2
vec_r2 rotate_minus1 ROTATE vec_r3
mstrip_3 vec_r3 MULTIPLY prod_3
prod_0 prod_1 ADD sum_0
sum_0 prod_2 ADD sum_1
sum_1 prod_3 ADD output_vec
"""

In [ ]:
sheep_client.new_job()
sheep_client.set_context("SEAL_BFV")
sheep_client.set_input_type("int16_t")
sheep_client.set_circuit_text(new_circuit)

We set some of the inputs to be the same as in the naive case above:

In [ ]:
const_input_vals = {"rotate_minus1": -1}
input_vals = {"vec": [1,2,3,4]}

For the matrix though, we take diagonal strips:

In [ ]:
input_vals["mstrip_0"] = [1,6,11,16]
input_vals["mstrip_1"] = [2,7,12,13]
input_vals["mstrip_2"] = [3,8,9,14]
input_vals["mstrip_3"] = [4,5,10,15]
sheep_client.set_inputs(input_vals)
sheep_client.set_const_inputs(const_input_vals)
sheep_client.run_job()
sheep_client.get_results()

We got the right answer again!  
Lets compare with HElib again:

In [ ]:
benchmark_utils.upload_results("matrix_vector_mult_4x4x4_fast")
sheep_client.new_job()
sheep_client.set_context("HElib_Fp")
sheep_client.set_input_type("int16_t")
sheep_client.set_parameters({"Levels":10})
sheep_client.set_circuit_text(new_circuit)
sheep_client.set_inputs(input_vals)
sheep_client.set_const_inputs(const_input_vals)
sheep_client.run_job()
benchmark_utils.upload_results("matrix_vector_mult_4x4x4_fast")

In [ ]:
rows = session.query(BenchmarkMeasurement).filter_by(circuit_name="matrix_vector_mult_4x4x4_fast").all()
df_dict = {"Context": [], "Gate": [], "Time": []}
for row in rows:
    timingdict = benchmark_utils.timing_per_gate_type(row.timings, new_circuit)
    for k,v in timingdict.items():
        df_dict["Context"].append(row.context)
        df_dict["Gate"].append(k)
        df_dict["Time"].append(v)
df = pd.DataFrame(df_dict)
pivot_df = df.pivot(index='Context', columns='Gate', values='Time')
pivot_df.plot(kind='bar',stacked=True)

## Using SEAL_CKKS with floating point numbers

The CKKS algorithm allows us to handle floating point numbers.  Lets do a simple addition:

In [ ]:
sheep_client.new_job()
sheep_client.set_context("SEAL_CKKS")
sheep_client.set_input_type("double")
sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))
sheep_client.set_inputs({"input_0":[1.1,3.3,4.4],"input_1":[2.1,2.1,2.1]})
sheep_client.run_job()
sheep_client.get_results()

... and complex numbers - we input the values as strings in the format "a.b+c.di" (spaces are tolerated, decimal points can be included or omitted, but you MUST put the "i" at the end of the imaginary part).

In [ ]:
sheep_client.new_job()
sheep_client.set_context("SEAL_CKKS")
sheep_client.set_input_type("complex")
sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))
sheep_client.set_inputs({"input_0":["1.1+3.3i"],"input_1":["2.1-3.3i"]})
sheep_client.run_job()
sheep_client.get_results()

## Summary

- SHEEP is an open-source project
- Looking for collaborators and users
- Let's integrate your library!

## The SHEEP Project

#### http://turing.ac.uk/research/research-projects/evaluating-homomorphic-encryption
#### Available from <br />  https://github.com/alan-turing-institute/sheep

